In [41]:
import vosk
import wave
import json
from deep_translator import GoogleTranslator
from vosk import SetLogLevel
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter
from melo.api import TTS
SetLogLevel(-1)

In [49]:
russian_file_path = 'my_name.wav'

In [50]:
import IPython.display as ipd

ipd.display(ipd.Audio(russian_file_path))

In [58]:
model = vosk.Model("vosk-model-small-ru-0.22")
file = wave.open(russian_file_path, "rb")

rec = vosk.KaldiRecognizer(model, file.getframerate())

while True:
    data = file.readframes(100000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        result = rec.Result()
        result_json = json.loads(result)

text = json.loads(rec.FinalResult())['text']
translate = GoogleTranslator(source='ru', target='en').translate(text)

In [59]:
translate

'Hello everyone, my name is Ilya Drobysheva, I am studying in the applied mathematics and computer science program'

In [65]:
ckpt_converter = 'checkpoints_v2/converter'
device = "cuda" if torch.cuda.is_available() else "cpu"
tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

target_se, audio_name = se_extractor.get_se(russian_file_path, tone_color_converter, vad=False)

src_path = f'tmp.wav'

# Speed is adjustable
speed = 1.0

model = TTS(language='EN', device=device)
speaker_ids = model.hps.data.spk2id

speaker_key = 'EN_INDIA'  #EN-BR, EN_INDIA, EN-AU, EN-Default, EN-US
speaker_id = speaker_ids[speaker_key]
speaker_key = speaker_key.lower().replace('_', '-')
source_se = torch.load(f'checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_location=device)
model.tts_to_file(translate, speaker_id, src_path, speed=speed)
save_path = f'result_{speaker_key}.wav'

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path,
    src_se=source_se,
    tgt_se=target_se,
    output_path=save_path,
    message=encode_message)
!rm tmp.wav

Loaded checkpoint 'checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []
OpenVoice version: v2
 > Text split to sentences.
Hello everyone, my name is Ilya Drobysheva, I am studying in the applied mathematics and computer science program
 > ===========================


100%|██████████| 1/1 [00:02<00:00,  2.50s/it]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [61]:
import IPython.display as ipd

ipd.display(ipd.Audio(russian_file_path))

In [62]:
ipd.display(ipd.Audio(f'result_{speaker_key}.wav')) # американский

In [64]:
ipd.display(ipd.Audio(f'result_{speaker_key}.wav')) # дефолтный

In [66]:
ipd.display(ipd.Audio(f'result_{speaker_key}.wav')) # индийский